# Homework and bake-off: Sentiment analysis

In [1]:
__author__ = "Christopher Potts"
__version__ = "CS224u, Stanford, Spring 2021"

## Contents

1. [Overview](#Overview)
1. [Methodological note](#Methodological-note)
1. [Set-up](#Set-up)
1. [Train set](#Train-set)
1. [Dev sets](#Dev-sets)
1. [A softmax baseline](#A-softmax-baseline)
1. [RNNClassifier wrapper](#RNNClassifier-wrapper)
1. [Error analysis](#Error-analysis)
1. [Homework questions](#Homework-questions)
  1. [Token-level differences [1 point]](#Token-level-differences-[1-point])
  1. [Training on some of the bakeoff data [1 point]](#Training-on-some-of-the-bakeoff-data-[1-point])
  1. [A more powerful vector-averaging baseline [2 points]](#A-more-powerful-vector-averaging-baseline-[2-points])
  1. [BERT encoding [2 points]](#BERT-encoding-[2-points])
  1. [Your original system [3 points]](#Your-original-system-[3-points])
1. [Bakeoff [1 point]](#Bakeoff-[1-point])
1. [Submission Instruction](#Submission-Instruction)

## Overview

This homework and associated bakeoff are devoted to supervised sentiment analysis using the ternary (positive/negative/neutral) version of the Stanford Sentiment Treebank (SST-3) as well as a new dev/test dataset drawn from restaurant reviews. Our goal in introducing the new dataset is to push you to create a system that performs well in both the movie and restaurant domains.

The homework questions ask you to implement some baseline system, and the bakeoff challenge is to define a system that does well at both the SST-3 test set and the new restaurant test set. Both are ternary tasks, and our central bakeoff score is the mean of the macro-FI scores for the two datasets. This assigns equal weight to all classes and datasets regardless of size.

The SST-3 test set will be used for the bakeoff evaluation. This dataset is already publicly distributed, so we are counting on people not to cheat by developing their models on the test set. You must do all your development without using the test set at all, and then evaluate exactly once on the test set and turn in the results, with no further system tuning or additional runs. __Much of the scientific integrity of our field depends on people adhering to this honor code__. 

One of our goals for this homework and bakeoff is to encourage you to engage in __the basic development cycle for supervised models__, in which you

1. Design a new system. We recommend starting with something simple.
1. Use `sst.experiment` to evaluate your system, using random train/test splits initially.
1. If you have time, compare your system with others using `sst.compare_models` or `utils.mcnemar`. (For discussion, see [this notebook section](sst_02_hand_built_features.ipynb#Statistical-comparison-of-classifier-models).)
1. Return to step 1, or stop the cycle and conduct a more rigorous evaluation with hyperparameter tuning and assessment on the `dev` set.

[Error analysis](#Error-analysis) is one of the most important methods for steadily improving a system, as it facilitates a kind of human-powered hill-climbing on your ultimate objective. Often, it takes a careful human analyst just a few examples to spot a major pattern that can lead to a beneficial change to the feature representations.

## Methodological note

You don't have to use the experimental framework defined below (based on `sst`). The only constraint we need to place on your system is that it must have a `predict_one` method that can map directly from an example text to a prediction, and it must be able to make predictions without having any information beyond the text. (For example, it can't depend on knowing which task the text comes from.) See [the bakeoff section below](#Bakeoff-[1-point]) for examples of functions that conform to this specification.

## Set-up

See [the first notebook in this unit](sst_01_overview.ipynb#Set-up) for set-up instructions.

In [2]:
from collections import Counter
import numpy as np
import os
import pandas as pd
from sklearn.linear_model import LogisticRegression
import torch.nn as nn

from torch_rnn_classifier import TorchRNNClassifier
from torch_tree_nn import TorchTreeNN
import sst
import utils

In [3]:
import torch
import torch.nn as nn

In [4]:
SST_HOME = os.path.join('data', 'sentiment')

## Train set

Our primary train set is the SST-3 train set:

In [5]:
sst_train = sst.train_reader(SST_HOME)

In [6]:
sst_train.shape[0]

8544

This is the train set we will use for all the regular homework questions. You are welcome to bring in new datasets for your original system. You are also free to add `include_subtrees=True`. This is very likely to lead to better systems, but it substantially increases the overall size of the dataset (from 8,544 examples to 159,274), which will in turn substantially increase the time it takes to run experiments.

See [this notebook](sst_01_overview.ipynb) for additional details of this dataset.

## Dev sets

We have two development set. SST3-dev consists of sentences from movie reviews, just like SST-3 train:

In [7]:
sst_dev = sst.dev_reader(SST_HOME)

Our new bakeoff dev set consists of sentences from restaurant reviews:

In [8]:
bakeoff_dev = sst.bakeoff_dev_reader(SST_HOME)

In [9]:
bakeoff_dev.sample(3, random_state=1).to_dict(orient='records')

[{'example_id': 57,
  'sentence': 'I would recommend that you make reservations in advance.',
  'label': 'neutral',
  'is_subtree': 0},
 {'example_id': 590,
  'sentence': 'We were welcomed warmly.',
  'label': 'positive',
  'is_subtree': 0},
 {'example_id': 1968,
  'sentence': 'We have been to Oceanaire twice in the last 6 weeks.',
  'label': 'neutral',
  'is_subtree': 0}]

Here is the label distribution:

In [10]:
bakeoff_dev.label.value_counts()

neutral     1019
positive     777
negative     565
Name: label, dtype: int64

The label distribution for the corresponding test set is similar to this.

## A softmax baseline

This example is here mainly as a reminder of how to use our experimental framework with linear models:

In [11]:
def unigrams_phi(text):
    return Counter(text.split())

Thin wrapper around `LogisticRegression` for the sake of `sst.experiment`:

In [12]:
def fit_softmax_classifier(X, y):
    mod = LogisticRegression(
        fit_intercept=True,
        solver='liblinear',
        multi_class='ovr')
    mod.fit(X, y)
    return mod

The experimental run with some notes:

In [13]:
softmax_experiment = sst.experiment(
    sst.train_reader(SST_HOME),   # Train on any data you like except SST-3 test!
    unigrams_phi,                 # Free to write your own!
    fit_softmax_classifier,       # Free to write your own!
    assess_dataframes=[sst_dev, bakeoff_dev]) # Free to change this during development!

Assessment dataset 1
              precision    recall  f1-score   support

    negative      0.628     0.689     0.657       428
     neutral      0.343     0.153     0.211       229
    positive      0.629     0.750     0.684       444

    accuracy                          0.602      1101
   macro avg      0.533     0.531     0.518      1101
weighted avg      0.569     0.602     0.575      1101

Assessment dataset 2
              precision    recall  f1-score   support

    negative      0.272     0.690     0.390       565
     neutral      0.429     0.113     0.179      1019
    positive      0.409     0.346     0.375       777

    accuracy                          0.328      2361
   macro avg      0.370     0.383     0.315      2361
weighted avg      0.385     0.328     0.294      2361

Mean of macro-F1 scores: 0.416


`softmax_experiment` contains a lot of information that you can use for error analysis; see [this section below](#Error-analysis) for starter code.

## RNNClassifier wrapper

This section illustrates how to use `sst.experiment` with `TorchRNNClassifier`.

To featurize examples for an RNN, we can just get the words in order, letting the model take care of mapping them into an embedding space.

In [14]:
def rnn_phi(text):
    return text.split()

The model wrapper gets the vocabulary using `sst.get_vocab`. If you want to use pretrained word representations in here, then you can have `fit_rnn_classifier` build that space too; see [this notebook section for details](sst_03_neural_networks.ipynb#Pretrained-embeddings). See also [torch_model_base.py](torch_model_base.py) for details on the many optimization parameters that `TorchRNNClassifier` accepts.

In [15]:
def fit_rnn_classifier(X, y):
    sst_glove_vocab = utils.get_vocab(X, mincount=2)
    mod = TorchRNNClassifier(
        sst_glove_vocab,
        early_stopping=True)
    mod.fit(X, y)
    return mod

In [16]:
rnn_experiment = sst.experiment(
    sst.train_reader(SST_HOME),
    rnn_phi,
    fit_rnn_classifier,
    vectorize=False,  # For deep learning, use `vectorize=False`.
    assess_dataframes=[sst_dev, bakeoff_dev])

Stopping after epoch 54. Validation score did not improve by tol=1e-05 for more than 10 epochs. Final error is 0.6502929627895355

Assessment dataset 1
              precision    recall  f1-score   support

    negative      0.618     0.570     0.593       428
     neutral      0.292     0.310     0.301       229
    positive      0.661     0.689     0.675       444

    accuracy                          0.564      1101
   macro avg      0.524     0.523     0.523      1101
weighted avg      0.567     0.564     0.565      1101

Assessment dataset 2
              precision    recall  f1-score   support

    negative      0.308     0.342     0.324       565
     neutral      0.419     0.428     0.423      1019
    positive      0.394     0.351     0.371       777

    accuracy                          0.382      2361
   macro avg      0.374     0.374     0.373      2361
weighted avg      0.384     0.382     0.382      2361

Mean of macro-F1 scores: 0.448


## Error analysis

This section begins to build an error-analysis framework using the dicts returned by `sst.experiment`. These have the following structure:

```
'model': trained model
'phi': the feature function used
'train_dataset':
   'X': feature matrix
   'y': list of labels
   'vectorizer': DictVectorizer,
   'raw_examples': list of raw inputs, before featurizing   
'assess_datasets': list of datasets, each with the same structure as the value of 'train_dataset'
'predictions': list of lists of predictions on the assessment datasets
'metric': `score_func.__name__`, where `score_func` is an `sst.experiment` argument
'score': the `score_func` score on the each of the assessment dataasets
```
The following function just finds mistakes, and returns a `pd.DataFrame` for easy subsequent processing:

In [17]:
def find_errors(experiment):
    """Find mistaken predictions.

    Parameters
    ----------
    experiment : dict
        As returned by `sst.experiment`.

    Returns
    -------
    pd.DataFrame

    """
    dfs = []
    for i, dataset in enumerate(experiment['assess_datasets']):
        df = pd.DataFrame({
            'raw_examples': dataset['raw_examples'],
            'predicted': experiment['predictions'][i],
            'gold': dataset['y']})
        df['correct'] = df['predicted'] == df['gold']
        df['dataset'] = i
        dfs.append(df)
    return pd.concat(dfs)

In [18]:
softmax_analysis = find_errors(softmax_experiment)

In [19]:
rnn_analysis = find_errors(rnn_experiment)

Here we merge the sotmax and RNN experiments into a single DataFrame:

In [20]:
analysis = softmax_analysis.merge(
    rnn_analysis, left_on='raw_examples', right_on='raw_examples')

analysis = analysis.drop('gold_y', axis=1).rename(columns={'gold_x': 'gold'})

The following code collects a specific subset of examples; small modifications to its structure will give you different interesting subsets:

In [21]:
# Examples where the softmax model is correct, the RNN is not,
# and the gold label is 'positive'

error_group = analysis[
    (analysis['predicted_x'] == analysis['gold'])
    &
    (analysis['predicted_y'] != analysis['gold'])
    &
    (analysis['gold'] == 'positive')
]

In [23]:
for ex in error_group['raw_examples'].sample(5, random_state=1):
    print("="*70)
    print(ex)

I just loved every minute of this film .
I had heard good things.
Instead of a hyperbolic beat-charged urban western , it 's an unpretentious , sociologically pointed slice of life .
This tenth feature is a big deal , indeed -- at least the third-best , and maybe even a notch above the previous runner-up , Nicholas Meyer 's Star Trek VI : The Undiscovered Country .
Ringside is one of the better steakhouses in the Northwestexcellent food,excellent service,and an excellent ambience!


## Homework questions

Please embed your homework responses in this notebook, and do not delete any cells from the notebook. (You are free to add as many cells as you like as part of your responses.)

### Token-level differences [1 point]

We can begin to get a sense for how our two dev sets differ by considering the most frequent tokens from each. This question asks you to begin such analysis.

Your task: write a function `get_token_counts` that, given a `pd.DataFrame` in the format of our datasets, tokenizes the example sentences based on whitespace and creates a count distribution over all of the tokens. The function should return a `pd.Series` sorted by frequency; if you create a count dictionary `d`, then `pd.Series(d).sort_values(ascending=False)` will give you what you need.

In [24]:
def get_token_counts(df):
    
    ##### YOUR CODE HERE
    from itertools import chain

    df['tokenized'] = df['sentence'].apply(lambda x: x.split())
    token_list = Counter(chain(*df['tokenized']))
    return pd.Series(token_list).sort_values(ascending=False)

In [25]:
def test_get_token_counts(func):
    df = pd.DataFrame([
        {'sentence': 'a a b'},
        {'sentence': 'a b a'},
        {'sentence': 'a a a b.'}])
    result = func(df)
    for token, expected in (('a', 7), ('b', 2), ('b.', 1)):
        actual = result.loc[token]
        assert actual == expected, \
            "For token {}, expected {}; got {}".format(
            token, expected, actual)

In [26]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    test_get_token_counts(get_token_counts)

As you develop your original system, you might review these results. The two dev sets have different vocabularies and different low-level encoding details that are sure to impact model performance, especially when one considers that the train set is like `sst_dev` in all these respects. For additional discussion, see [this notebook section](sst_01_overview.ipynb#Tokenization).

### Training on some of the bakeoff data [1 point]

We have so far presented the bakeoff dev set as purely for evaluation. Since the train set consists entirely of SST-3 data, this makes the bakeoff split especially challenging. We might be able to reduce the challenging by adding some of the bakeoff dev set to the train set, keeping some of it for evaluation. The current question asks to begin explore the effects of such training.

Your task: write a function `run_mixed_training_experiment`. The function should:

1. Take as inputs (a) a model training wrapper like `fit_softmax_classifier` and (b) an integer `bakeoff_train_size` specifying the number of examples from `bakeoff_dev` that should be included in the train set.
1. Split `bakeoff_dev` so that the first `bakeoff_train_size` examples are in the train set and the rest are used for evaluation.
1. Use `sst.experiment` with the user-supplied model training wrapper, `unigram_phi` as defined above, and a train set that consists of SST-3 train and the train portion of `bakeoff_dev` as defined in step 2. The value of `assess_dataframes` should be a list consisting of the SST-3 dev set and the evaluation portion of `bakeoff_dev` as defined in step 2.
1. Return the return value of `sst.experiment`.

The function `test_run_mixed_training_experiment` will help you iterate to the required design.

In [27]:
def run_mixed_training_experiment(wrapper_func, bakeoff_train_size):
    ##### YOUR CODE HERE
    train_bakeoff_set, eval_bakeoff_set = bakeoff_dev[:bakeoff_train_size], bakeoff_dev[bakeoff_train_size:]
    train_set = pd.concat([sst_train,train_bakeoff_set])
    experiment = sst.experiment(
        train_set,
        unigrams_phi,
        wrapper_func,
        vectorize=True,  # For deep learning, use `vectorize=False`.
        assess_dataframes=[sst_dev, eval_bakeoff_set])
    return experiment



In [28]:
def test_run_mixed_training_experiment(func):
    bakeoff_train_size = 1000
    experiment = func(fit_softmax_classifier, bakeoff_train_size)

    assess_size = len(experiment['assess_datasets'])
    assert len(experiment['assess_datasets']) == 2, \
        ("The evaluation should be done on two datasets: "
         "SST3 and part of the bakeoff dev set. "
         "You have {} datasets.".format(assess_size))

    bakeoff_test_size = bakeoff_dev.shape[0] - bakeoff_train_size
    expected_eval_examples = bakeoff_test_size + sst_dev.shape[0]
    eval_examples = sum(len(d['raw_examples']) for d in experiment['assess_datasets'])
    assert expected_eval_examples == eval_examples, \
        "Expected {} evaluation examples; got {}".format(
        expected_eval_examples, eval_examples)

In [29]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    test_run_mixed_training_experiment(run_mixed_training_experiment)

Assessment dataset 1
              precision    recall  f1-score   support

    negative      0.627     0.671     0.648       428
     neutral      0.319     0.162     0.214       229
    positive      0.638     0.757     0.692       444

    accuracy                          0.599      1101
   macro avg      0.528     0.530     0.518      1101
weighted avg      0.567     0.599     0.576      1101

Assessment dataset 2
              precision    recall  f1-score   support

    negative      0.471     0.412     0.440       320
     neutral      0.588     0.590     0.589       612
    positive      0.503     0.548     0.525       429

    accuracy                          0.535      1361
   macro avg      0.521     0.517     0.518      1361
weighted avg      0.534     0.535     0.534      1361

Mean of macro-F1 scores: 0.518


### A more powerful vector-averaging baseline [2 points]

In [Distributed representations as features](sst_03_neural_networks.ipynb#Distributed-representations-as-features), we looked at a baseline for the ternary SST-3 problem in which each example is modeled as the mean of its GloVe representations. A `LogisticRegression` model was used for prediction. A neural network might do better with these representations, since there might be complex relationships between the input feature dimensions that a linear classifier can't learn. To address this question, we want to get set up to run the experiment with a shallow neural classifier. 

Your task: write and submit a model wrapper function around `TorchShallowNeuralClassifier`. This function should implement hyperparameter search according to this specification:

* Set `early_stopping=True` for all experiments.
* Using 3-fold cross-validation, exhaustively explore this set of hyperparameter combinations:
  * The hidden dimensionality at 50, 100, and 200.
  * The hidden activation function as `nn.Tanh()` and `nn.ReLU()`.
* For all other parameters to `TorchShallowNeuralClassifier`, use the defaults.

See [this notebook section](sst_02_hand_built_features.ipynb#Hyperparameter-search) for examples. You are not required to run a full evaluation with this function using `sst.experiment`, but we assume you will want to.

We're not evaluating the quality of your model. (We've specified the protocols completely, but there will still be variation in the results.) However, the primary goal of this question is to get you thinking more about this strong baseline feature representation scheme for SST-3, so we're sort of hoping you feel compelled to try out variations on your own.

In [30]:
from torch_shallow_neural_classifier import TorchShallowNeuralClassifier

def fit_shallow_neural_classifier_with_hyperparameter_search(X, y):
    
    ##### YOUR CODE HERE
    basemod = TorchShallowNeuralClassifier(
        early_stopping=True)

    param_grid = {
        'hidden_dim': [50, 100, 200],
        'hidden_activation':[nn.Tanh(),nn.ReLU()]}

    bestmod = utils.fit_classifier_with_hyperparameter_search(
        X, y, basemod, cv=3, param_grid=param_grid)
    
    return bestmod



### BERT encoding [2 points]

We might hypothesize that encoding our examples with BERT will yield improvements over the GloVe averaging method explored in the previous question, since BERT implements a much more complex and data-driven function for this kind of combination. This question asks you to begin exploring this general hypothesis.

Your task: write a function `hf_cls_phi` that uses Hugging Face functionality to encode individual examples with BERT and returns the final output representation above the [CLS] token.

You are not required to evaluate this feature function, but it is easy to do so with `sst.experiment` and `vectorize=False` (since your feature function directly encodes every example as a vector). Your code should also be a natural basis for even more powerful approaches – for example, it might be even better to pool all the output states rather than using just the first output state. Another option is [fine-tuning](finetuning.ipynb).

In [32]:
from transformers import BertModel, BertTokenizer
import vsm

# Instantiate a Bert model and tokenizer based on `bert_weights_name`:
bert_weights_name = 'bert-base-uncased'
##### YOUR CODE HERE
bert_tokenizer = BertTokenizer.from_pretrained(bert_weights_name)
bert_model = BertModel.from_pretrained(bert_weights_name)

def hf_cls_phi(text):
    # Get the ids. `vsm.hf_encode` will help; be sure to
    # set `add_special_tokens=True`.
    ##### YOUR CODE HERE 
    ids = vsm.hf_encode(text, bert_tokenizer, add_special_tokens=True)

    # Get the BERT representations. `vsm.hf_represent` will help:
    ##### YOUR CODE HERE
    reps = vsm.hf_represent(ids, bert_model, layer=-1)

    
    # Index into `reps` to get the representation above [CLS].
    # The shape of `reps` should be (1, n, 768), where n is the
    # number of tokens. You need the 0th element of the 2nd dim:
    ##### YOUR CODE HERE
    cls_rep = reps[:,0,:].squeeze()

    # These conversions should ensure that you can work with the
    # representations flexibly. Feel free to change the variable
    # name:
    return cls_rep.cpu().numpy()

In [33]:
def test_hf_cls_phi(func):
    rep = func("Just testing!")

    expected_shape = (768,)
    result_shape = rep.shape
    assert rep.shape == (768,), \
        "Expected shape {}; got {}".format(
        expected_shape, result_shape)

    # String conversion to avoid precision errors:
    expected_first_val = str(0.1709)
    result_first_val = "{0:.04f}".format(rep[0])

    assert expected_first_val == result_first_val, \
        ("Unexpected representation values. Expected the "
        "first value to be {}; got {}".format(
            expected_first_val, result_first_val))

In [34]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    test_hf_cls_phi(hf_cls_phi)

Note: encoding all of SST-3 train (no subtrees) takes about 11 minutes on my 2015 iMac, CPU only (32GB).

### Your original system [3 points]

Your task is to develop an original model for the SST-3 problem and our new bakeoff dataset. There are many options. If you spend more than a few hours on this homework problem, you should consider letting it grow into your final project! Here are some relatively manageable ideas that you might try:

1. We didn't systematically evaluate the `bidirectional` option to the `TorchRNNClassifier`. Similarly, that model could be tweaked to allow multiple LSTM layers (at present there is only one), and you could try adding layers to the classifier portion of the model as well.

1. We've already glimpsed the power of rich initial word representations, and later in the course we'll see that smart initialization usually leads to a performance gain in NLP, so you could perhaps achieve a winning entry with a simple model that starts in a great place.

1. Our [practical introduction to contextual word representations](finetuning.ipynb) covers pretrained representations and interfaces that are likely to boost the performance of any system.

We want to emphasize that this needs to be an __original__ system. It doesn't suffice to download code from the Web, retrain, and submit. You can build on others' code, but you have to do something new and meaningful with it. See the course website for additional guidance on how original systems will be evaluated.

In the cell below, please provide a brief technical description of your original system, so that the teaching team can gain an understanding of what it does. This will help us to understand your code and analyze all the submissions to identify patterns and strategies.  We also ask that you report the best score your system got during development (your best average of macro-F1 scores), just to help us understand how systems performed overall.

In [6]:
# PLEASE MAKE SURE TO INCLUDE THE FOLLOWING BETWEEN THE START AND STOP COMMENTS:
#   1) Textual description of your system.
#   2) The code for your original system.
#   3) The score achieved by your system in place of MY_NUMBER.
#        With no other changes to that line.
#        You should report your score as a decimal value <=1.0
# PLEASE MAKE SURE NOT TO DELETE OR EDIT THE START AND STOP COMMENTS

# NOTE: MODULES, CODE AND DATASETS REQUIRED FOR YOUR ORIGINAL SYSTEM
# SHOULD BE ADDED BELOW THE 'IS_GRADESCOPE_ENV' CHECK CONDITION. DOING
# SO ABOVE THE CHECK MAY CAUSE THE AUTOGRADER TO FAIL.

# START COMMENT: Enter your system description in this cell.
# The system is developed with the use of pytorch-lightning framework. 
# To execute this cell you need to run in your terminal: $pip install pytorch-lightning==1.4.5
# The system consists of 3 parts: custom tokenizer, dataset for Neural Network training preparation and LSTM model.
# There are no pretrained embeddings used. The LSTM is bidirectional with 1 layer and its hidden size is 100. 
# The embedding size is 300. Batch size is 64. AdamW is used for optimization. 
# My peak score was: 0.53
if 'IS_GRADESCOPE_ENV' not in os.environ:
    import torch
    import torch.nn as nn
    import pandas as pd
    import numpy as np
    import os
    import sst
    import pytorch_lightning as pl
    from torch.utils.data.dataloader import DataLoader
    from pytorch_lightning.callbacks.early_stopping import EarlyStopping
    from pytorch_lightning.callbacks import ModelCheckpoint
    from sklearn.metrics import f1_score


    class SentimentTokenizer:
        def __init__(self, corpus):
            self.PAD_TOKEN = "<PAD>"
            self.SOS_TOKEN = "<s>"
            self.EOS_TOKEN = "<s/>"
            self.OOV_TOKEN = "<UNK>"
            self.idx2tok = [self.PAD_TOKEN, self.SOS_TOKEN, self.EOS_TOKEN, self.OOV_TOKEN]
            tokens = corpus.lower().split()
            words = set(tokens)
            self.idx2tok += list(words)
            self.tok2idx = {t: ix for ix, t in enumerate(self.idx2tok)}

        def get_vocab_size(self):
            return len(self.idx2tok)

        def encode(self, text):
            tokens = [self.SOS_TOKEN] + text.lower().split() + [self.EOS_TOKEN]
            encoded = [
                self.tok2idx[t]
                if t in self.tok2idx.keys()
                else self.tok2idx[self.OOV_TOKEN]
                for t in tokens
            ]
            return torch.LongTensor(encoded)

        def decode(self, encoded):
            decoded = [self.idx2tok[ix] for ix in encoded]
            decoded_text = " ".join(decoded)
            return decoded_text

        def get_start_token_idx(self):
            return self.tok2idx[self.SOS_TOKEN]

        def get_pad_token_idx(self):
            return self.tok2idx[self.PAD_TOKEN]


    class SentimentDataset(torch.utils.data.Dataset):
        def __init__(self, tokenizer, df, label2idx):
            super().__init__()
            self.df = df
            labels = list(set(self.df["label"]))
            self.n_classes = len(labels)
            self.label2idx = label2idx
            self.tokenizer = tokenizer

        def collate_fn(self, batch):
            batch_elements = list(zip(*batch))
            X = batch_elements[0]
            pad_val = self.tokenizer.get_pad_token_idx()
            X = torch.nn.utils.rnn.pad_sequence(X, batch_first=True, padding_value=pad_val)
            y = torch.LongTensor(batch_elements[1])
            return X, y

        def __len__(self):
            return len(self.df)

        def __getitem__(self, idx):
            x = self.df.iloc[idx]["sentence"]
            y = self.df.iloc[idx]["label"]

            x_encoded = self.tokenizer.encode(x)
            y_encoded = self.label2idx[y]

            return x_encoded, y_encoded

        def get_n_classes(self):
            return self.n_classes


    class BidirectionalRNN(pl.LightningModule):
        def __init__(
            self,
            vocab_size,
            emb_dim,
            hidden_dim,
            n_classes,
            bidirectional,
            train_set=None,
            val_set=None,
            num_layers=1,
        ):
            super().__init__()
            self.embedding = nn.Embedding(vocab_size, emb_dim)
            self.rnn = nn.LSTM(
                emb_dim,
                hidden_dim,
                bidirectional=bidirectional,
                batch_first=True,
                num_layers=num_layers,
            )
            self.dropout = nn.Dropout()
            if bidirectional:
                self.fc = nn.Linear(hidden_dim * 2, n_classes)
            else:
                self.fc = nn.Linear(hidden_dim, n_classes)
            self.criterion = nn.CrossEntropyLoss()
            self.train_set = train_set
            self.val_set = val_set

        def forward(self, X):
            embedded = self.embedding(X)
            output, (hidden, _) = self.rnn(embedded)
            hidden = torch.cat([hidden[0], hidden[1]], dim=1)
            hidden = self.dropout(hidden)
            y = torch.softmax(self.fc(hidden), dim=1)
            return y

        def training_step(self, batch, batch_idx):
            x, y = batch
            output = self.forward(x)
            preds = torch.argmax(output, dim=1)
            loss = self.criterion(output, y)
            self.log("train_loss", loss)
            return {"loss": loss}

        def validation_step(self, batch, batch_idx):
            x, y = batch
            output = self.forward(x)
            preds = torch.argmax(output, dim=1)
            f1 = f1_score(y, preds, average="macro")
            loss = self.criterion(output, y)
            return {"loss": loss, "f1_macro": f1}

        def validation_epoch_end(self, outputs):
            loss = torch.stack([x["loss"] for x in outputs]).mean()
            f1 = np.stack([x["f1_macro"] for x in outputs]).mean()
            self.log("val_loss", loss)
            self.log("f1_macro", f1)

            return {"f1_macro": f1}

        def configure_optimizers(self):
            return torch.optim.AdamW(self.parameters())

        def train_dataloader(self):
            return DataLoader(
                self.train_set,
                batch_size=64,
                shuffle=True,
                collate_fn=self.train_set.collate_fn,
                num_workers=10,
            )

        def val_dataloader(self):
            return DataLoader(
                self.val_set,
                batch_size=64,
                shuffle=False,
                collate_fn=self.val_set.collate_fn,
                num_workers=10,
            )

    def train_evaluate_model():
        label2idx = {"positive": 2, "neutral": 1, "negative": 0}
        SST_HOME = os.path.join("data", "sentiment")

        sst_train = sst.train_reader(SST_HOME)
        bakeoff_dev = sst.bakeoff_dev_reader(SST_HOME)
        bakeoff_train, bakeoff_eval = bakeoff_dev[:1000], bakeoff_dev[1000:]
        train_full = pd.concat([sst_train, bakeoff_train])
        sst_dev = sst.dev_reader(SST_HOME)

        corpus = " ".join(list(train_full["sentence"]))
        tokenizer = SentimentTokenizer(corpus)

        train_dset = SentimentDataset(tokenizer, train_full, label2idx)
        bakeoff_val_dset = SentimentDataset(tokenizer, bakeoff_eval, label2idx)
        sst_val_dset = SentimentDataset(tokenizer, sst_dev, label2idx)

        n_classes = train_dset.get_n_classes()
        vocab_size = tokenizer.get_vocab_size()
        param_dict = {
            "embedding_size": 300,
            "hidden_size": 100,
            "bidirectional": True,
            "vocab_size": vocab_size,
        }
        model = BidirectionalRNN(
            vocab_size=param_dict["vocab_size"],
            emb_dim=param_dict["embedding_size"],
            hidden_dim=param_dict["hidden_size"],
            n_classes=n_classes,
            bidirectional=param_dict["bidirectional"],
            train_set=train_dset,
            val_set=sst_val_dset,
        )
        checkpoint_callback = ModelCheckpoint(
            monitor="val_loss", mode="min", save_last=True
        )
        early_stop_callback = EarlyStopping(monitor="val_loss", patience=10)
        trainer = pl.Trainer(
            callbacks=[checkpoint_callback, early_stop_callback],
            max_epochs=100,
            gradient_clip_val=1.0,
        )
        trainer.fit(model)
        print(f"Best model saved in: {checkpoint_callback.best_model_path}")
        model.load_from_checkpoint(
            checkpoint_callback.best_model_path,
            vocab_size=param_dict["vocab_size"],
            emb_dim=param_dict["embedding_size"],
            hidden_dim=param_dict["hidden_size"],
            n_classes=n_classes,
            bidirectional=param_dict["bidirectional"],
        )
        model.eval()
        sst_val = trainer.validate(
            model,
            DataLoader(
                sst_val_dset,
                batch_size=len(sst_val_dset),
                collate_fn=sst_val_dset.collate_fn,
                num_workers=10,
            ),
        )
        bakeoff_val = trainer.validate(
            model,
            DataLoader(
                bakeoff_val_dset,
                batch_size=len(bakeoff_val_dset),
                collate_fn=bakeoff_val_dset.collate_fn,
                num_workers=10,
            ),
        )

    
    
    train_evaluate_model()
    SST_HOME = os.path.join("data", "sentiment")
    sst_train = sst.train_reader(SST_HOME)
    bakeoff_dev = sst.bakeoff_dev_reader(SST_HOME)
    bakeoff_train, bakeoff_eval = bakeoff_dev[:1000], bakeoff_dev[1000:]
    train_full = pd.concat([sst_train, bakeoff_train])
    corpus = " ".join(list(train_full["sentence"]))
    tokenizer = SentimentTokenizer(corpus)
    label2idx = {"positive": 2, "neutral": 1, "negative": 0}
    idx2label = {ix: label for label, ix in label2idx.items()}
    model_checkpoint = "Untitled/NLU-16/checkpoints/epoch=3-step=599.ckpt"

    vocab_size = tokenizer.get_vocab_size()
    param_dict = {
        "embedding_size": 300,
        "hidden_size": 100,
        "bidirectional": True,
        "vocab_size": vocab_size,
    }
    my_model = BidirectionalRNN(
        vocab_size=param_dict["vocab_size"],
        emb_dim=param_dict["embedding_size"],
        hidden_dim=param_dict["hidden_size"],
        n_classes=len(idx2label),
        bidirectional=param_dict["bidirectional"],
    )
    my_model.load_from_checkpoint(
        model_checkpoint,
        vocab_size=param_dict["vocab_size"],
        emb_dim=param_dict["embedding_size"],
        hidden_dim=param_dict["hidden_size"],
        n_classes=len(idx2label),
        bidirectional=param_dict["bidirectional"],
    )
    


# STOP COMMENT: Please do not remove this comment.

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs

  | Name      | Type             | Params
-----------------------------------------------
0 | embedding | Embedding        | 5.5 M 
1 | rnn       | LSTM             | 321 K 
2 | dropout   | Dropout          | 0     
3 | fc        | Linear           | 603   
4 | criterion | CrossEntropyLoss | 0     
-----------------------------------------------
5.8 M     Trainable params
0         Non-trainable params
5.8 M     Total params
23.340    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

KeyboardInterrupt: 

## Bakeoff [1 point]

As we said above, the bakeoff evaluation data is the official SST test-set release and a new test set derived from the same sources and labeling methods as for `bakeoff_dev`.

For this bakeoff, you'll evaluate your original system from the above homework problem on these test sets. Our metric will be the mean of the macro-F1 values, which weights both datasets equally despite their differing sizes.

The central requirement for your system is that you have define a `predict_one` method for it that maps a text (str) directly to a label prediction – one of 'positive', 'negative', 'neutral'. If you used `sst.experiment` with `vectorize=True`, then the following function (for `softmax_experiment`) will be easy to adapt – you probably just need to change the variable `softmax_experiment` to the variable for your experiment output.

In [ ]:
def predict_one_softmax(text):
    # Singleton list of feature dicts:
    feats = [softmax_experiment['phi'](text)]
    # Vectorize to get a feature matrix:
    X = softmax_experiment['train_dataset']['vectorizer'].transform(feats)
    # Standard sklearn `predict` step:
    preds = softmax_experiment['model'].predict(X)
    # Be sure to return the only member of the predictions,
    # rather than the singleton list:
    return preds[0]

If you used an RNN like the one we demoed above, then featurization is a bit more straightforward:

In [ ]:
def predict_one_rnn(text):
    # List of tokenized examples:
    X = [rnn_experiment['phi'](text)]
    # Standard `predict` step on a list of lists of str:
    preds = rnn_experiment['model'].predict(X)
    # Be sure to return the only member of the predictions,
    # rather than the singleton list:
    return preds[0]

The following function is used to create the bakeoff submission file. Its arguments are your `predict_one` function and an output filename (str).

In [8]:
def create_bakeoff_submission(
        predict_one_func,
        output_filename='cs224u-sentiment-bakeoff-entry.csv'):

    bakeoff_test = sst.bakeoff_test_reader(SST_HOME)
    sst_test = sst.test_reader(SST_HOME)
    bakeoff_test['dataset'] = 'bakeoff'
    sst_test['dataset'] = 'sst3'
    df = pd.concat((bakeoff_test, sst_test))

    df['prediction'] = df['sentence'].apply(predict_one_func)

    df.to_csv(output_filename, index=None)

Thus, for example, the following will create a bake-off entry based on `predict_one_softmax`:

In [ ]:
def predict_one(text):
        my_model.eval()
        X = tokenizer.encode(text).unsqueeze(0)
        output = torch.argmax(my_model(X), dim=1).squeeze()
        label = idx2label[output.item()]
        return label

In [11]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    pass
    create_bakeoff_submission(predict_one)

This creates a file `cs224u-sentiment-bakeoff-entry.csv` in the current directory. That file should be uploaded as-is. Please do not change its name.

Only one upload per team is permitted, and you should do no tuning of your system based on what you see in our bakeoff prediction file – you should not study that file in anyway, beyond perhaps checking that it contains what you expected it to contain. The upload function will do some additional checking to ensure that your file is well-formed.

People who enter will receive the additional homework point, and people whose systems achieve the top score will receive an additional 0.5 points. We will test the top-performing systems ourselves, and only systems for which we can reproduce the reported results will win the extra 0.5 points.

Late entries will be accepted, but they cannot earn the extra 0.5 points.

## Submission Instruction

Submit the following files to gradescope submission

- Please do not change the file name as described below
- `hw_sentiment.ipynb` (this notebook)
- `cs224u-sentiment-bakeoff-entry.csv` (bake-off output)
